In [45]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7/python"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
os.environ["SPARK_HOME"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

In [46]:
from pyspark.sql import SparkSession 

In [47]:
spark = SparkSession.builder.appName("Trial").getOrCreate()

In [48]:
data = spark.read.csv('bitcoin.csv',header=True,inferSchema=True)

In [49]:
data.printSchema()

root
 |-- Timestamp: integer (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume_(BTC): double (nullable = true)
 |-- Volume_(Currency): double (nullable = true)
 |-- Weighted_Price: double (nullable = true)



In [50]:
data.show()

+----------+----+----+----+-----+------------+-----------------+--------------+
| Timestamp|Open|High| Low|Close|Volume_(BTC)|Volume_(Currency)|Weighted_Price|
+----------+----+----+----+-----+------------+-----------------+--------------+
|1325317920|4.39|4.39|4.39| 4.39|  0.45558087|     2.0000000193|          4.39|
|1325317980| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318040| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318100| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318160| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318220| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318280| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318340| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318400| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318460| NaN| NaN| NaN|  NaN|       

### Observation from the data table

There are a lot of NaN values. Need to check if the compete table is the same way. 

### Question 
Confirm the table has more than 1 meaningful occurances. 

In [51]:
# lets arrange the data in accending order of some column say High. if there are values in it they will be first. 


+----------+----+----+----+-----+------------+-----------------+--------------+
| Timestamp|Open|High| Low|Close|Volume_(BTC)|Volume_(Currency)|Weighted_Price|
+----------+----+----+----+-----+------------+-----------------+--------------+
|1327629480| 3.8| 3.8| 3.8|  3.8|        0.27|            1.026|           3.8|
|1328036700| 3.8| 3.8| 3.8|  3.8|         5.0|             19.0|           3.8|
|1329582120|4.14|4.14|4.14| 4.14|  2.24383721|     9.2894860494|          4.14|
|1329582180|4.14|4.14|4.14| 4.14|  4.58743279|     18.991971751|          4.14|
|1329415380|4.15|4.15|4.15| 4.15|  1.07011258|      4.440967207|          4.15|
|1329358140|4.17|4.17|4.17| 4.17|         4.0|            16.68|          4.17|
|1329601860|4.23|4.23|4.23| 4.23|         5.0|            21.15|          4.23|
|1329567300|4.24|4.24|4.21| 4.21| 11.48636364|     48.454590924|  4.2184447962|
|1329587760|4.25|4.25|4.25| 4.25|        20.0|             85.0|          4.25|
|1329567000|4.27|4.27|4.27| 4.27|       

There are values in the data set it is not completly empty. 

### Question 

Remove all the rows with NaN values.  

In [52]:


# drop all rows with empty values 

+----------+----+----+----+-----+------------+-----------------+--------------+
| Timestamp|Open|High| Low|Close|Volume_(BTC)|Volume_(Currency)|Weighted_Price|
+----------+----+----+----+-----+------------+-----------------+--------------+
|1325317920|4.39|4.39|4.39| 4.39|  0.45558087|     2.0000000193|          4.39|
|1325346600|4.39|4.39|4.39| 4.39|        48.0|           210.72|          4.39|
|1325350740| 4.5|4.57| 4.5| 4.57| 37.86229723|     171.38033753|  4.5264114983|
|1325350800|4.58|4.58|4.58| 4.58|         9.0|            41.22|          4.58|
|1325391360|4.58|4.58|4.58| 4.58|       1.502|          6.87916|          4.58|
|1325431680|4.84|4.84|4.84| 4.84|        10.0|             48.4|          4.84|
|1325457900| 5.0| 5.0| 5.0|  5.0|        10.1|             50.5|           5.0|
|1325534640| 5.0| 5.0| 5.0|  5.0|      19.048|            95.24|           5.0|
|1325591100|5.32|5.32|5.32| 5.32|  2.41917293|     12.869999988|          5.32|
|1325600520|5.14|5.14|5.14| 5.14|       

### Question 

Find the percentage empty rows.

percentage of non empty rows = (number of non-empty rows/number total number of rows)*100

percentage of empty rows = 100 - percentage of non empty rows

answer = 30.81

30.81469155866489

Notice the large amout of time it takes to compute this. It is because .count() is an action it is being performed right now. Moreover, the data set is quite large so it is taking time.

Let's check how large the data set actually is!

In [55]:

#number of rows in dataframe with no empty columns

2765819

In [56]:

#number of rows in the complete data set

3997697

### Handeling the timestamp column

Now lets try to convert the timestamp in more readable format.

### Question 
Convert the Timestamp column in normal datetime

hint : https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.from_unixtime

+-------------------+----+----+----+-----+------------+-----------------+--------------+
|          Timestamp|Open|High| Low|Close|Volume_(BTC)|Volume_(Currency)|Weighted_Price|
+-------------------+----+----+----+-----+------------+-----------------+--------------+
|2011-12-31 07:52:00|4.39|4.39|4.39| 4.39|  0.45558087|     2.0000000193|          4.39|
|2011-12-31 15:50:00|4.39|4.39|4.39| 4.39|        48.0|           210.72|          4.39|
|2011-12-31 16:59:00| 4.5|4.57| 4.5| 4.57| 37.86229723|     171.38033753|  4.5264114983|
|2011-12-31 17:00:00|4.58|4.58|4.58| 4.58|         9.0|            41.22|          4.58|
|2012-01-01 04:16:00|4.58|4.58|4.58| 4.58|       1.502|          6.87916|          4.58|
|2012-01-01 15:28:00|4.84|4.84|4.84| 4.84|        10.0|             48.4|          4.84|
|2012-01-01 22:45:00| 5.0| 5.0| 5.0|  5.0|        10.1|             50.5|           5.0|
|2012-01-02 20:04:00| 5.0| 5.0| 5.0|  5.0|      19.048|            95.24|           5.0|
|2012-01-03 11:45:00|

### Question 

Find the 90th percentile of weighted price

answer = '8541.8561284'

'8541.8561284'

### QUESTION

Find the date time when the weighted price was maximum

answer = '2017-12-17 12:24:00'

##### Notice:  
How this executes instantly. As there is now action

'2017-12-17 12:24:00'

##### Notice:
This takes time because it does the previous transformation first. We are using indexing to get to the timestamp in the firstrow

### Question

Find the total number of occurances when more than 10 Bitcoin was traded

ans = 

1749949

### Question 

Find the average quantity of bitcoin traded per year 

In [62]:
#import function to find the year form date
#import function to round off average



In [63]:
#Create a new dataframe with a year column


In [64]:
#find the averages of the yearly quantity


In [65]:
#show the yearly averages in a readable format



+----+-----+
|year| Mean|
+----+-----+
|2018| 7.77|
|2015|14.82|
|2013|15.73|
|2014|12.62|
|2019| 6.68|
|2012|21.33|
|2016| 5.66|
|2011|23.83|
|2017| 9.72|
+----+-----+

